In [4]:
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.optimizers import Adam
import numpy as np
import cv2

import glob
import os


In [6]:
size = 128

input = Input(shape = (size, size, 1))

conv1 = Conv2D(10, 4, activation='relu')(input)
dropout1 = Dropout(0.5)(conv1)
BatchNormalization()
pool1 = MaxPool2D(pool_size=(3, 3))(dropout1)

conv2 = Conv2D(20, 4, activation='relu')(pool1)
dropout2 = Dropout(0.5)(conv2)
BatchNormalization()
pool2 = MaxPool2D(pool_size=(3, 3))(dropout2)

flat = Flatten()(pool2)
hidden = Dense(128, activation='relu')(flat)
output = Dense(4, activation='softmax')(hidden)
model = Model(inputs=input, outputs=output)

In [7]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 125, 125, 10)      170       
                                                                 
 dropout_2 (Dropout)         (None, 125, 125, 10)      0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 41, 41, 10)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 38, 38, 20)        3220      
                                                                 
 dropout_3 (Dropout)         (None, 38, 38, 20)        0         
                                                           

## Train Datasets

In [1]:
datasets = "datasets_split/train/"

N = 1600

x_train = np.zeros((N, size, size, 1), 'float')
y_train = np.zeros((N), 'float')

count = 0

class_id = sorted(os.listdir(datasets))

for i, class_ in enumerate(class_id):
    datas = sorted(os.listdir(os.path.join(datasets, class_)))
    
    for data in datas:
        im = cv2.imread(os.path.join(datasets, class_, data), cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (size, size))
        x_train[count, :, :, 0] = im / 255.
        y_train[count] = i
        count += 1

y_train = to_categorical(y_train)

NameError: name 'np' is not defined

### Train Model

In [ ]:
h = model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=1)

In [ ]:
# save model
model.save("datasets_model.h5")

## Test Datasets

In [ ]:
datasets = "datasets_split/test/"

N = 400

x_test = np.zeros((N, size, size, 1), 'float')
y_test = np.zeros((N), 'float')

count = 0

class_id = sorted(os.listdir(datasets))

for i, class_ in enumerate(class_id):
    datas = sorted(os.listdir(os.path.join(datasets, class_)))
    
    for data in datas:
        im = cv2.imread(os.path.join(datasets, class_, data), cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (size, size))
        x_test[count, :, :, 0] = im / 255.
        y_test[count] = i
        count += 1

y_test = to_categorical(y_test)

## Validation Datasets

In [ ]:
datasets = "datasets_split/val/"

N = 400

x_val = np.zeros((N, size, size, 1), 'float')
y_val = np.zeros((N), 'float')

count = 0

class_id = sorted(os.listdir(datasets))

for i, class_ in enumerate(class_id):
    datas = sorted(os.listdir(os.path.join(datasets, class_)))
    
    for data in datas:
        im = cv2.imread(os.path.join(datasets, class_, data), cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im, (size, size))
        x_val[count, :, :, 0] = im / 255.
        y_val[count] = i
        count += 1

y_val = to_categorical(y_val)

### Train Model

In [ ]:
h = model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=1, validation_data=(x_val, y_val))

In [195]:
y_predic = model.predict(x_test)

prediction = y_predic.argmax(axis=1)


13/13 [==============================] - 0s 31ms/step


In [196]:
classifications = ['B', 'D', 'R', 'S']

data_test = 'datasets_split/test/'

images_test = []

for folder in os.listdir(data_test):
    folders = os.path.join(data_test, folder)
    
    if os.path.isdir(folders):
        
        for data in os.listdir(folders):
            
            if data.endswith('.jpg'):
                data_path = os.path.join(folders, data)
                images_test.append(data_path)

In [197]:
with open('result.txt', 'w') as f:
    
    for i in range(len(images_test)):
        image_name = os.path.basename(images_test[i])
        classification = classifications[prediction[i]]
        f.write("{}::{}".format(
            image_name, classification))

In [198]:
loss, accuracy = model.evaluate(x_test, y_test)

print(f'Total loss: {loss}')
print(f'Total accuracy: {accuracy}')

13/13 [==============================] - 0s 27ms/step - loss: 0.5534 - accuracy: 0.7900
Test loss: 0.5533946752548218
Test accuracy: 0.7900000214576721
